# 모델 매개변수 최적화

데이터에 매개변수를 최적화 -> 모델 학습, 검증, 테스트 진행

In [1]:
# 이전코드 재작성
# Dataset, DataLoader, 신경망 구성 코드 가져오기

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

## 하이퍼파라미터 조정
- epoch : 데이터셋을 반복하는 횟수
- batch_size : 매개변수 갱신 전 신경망에 전파된 데이터 샘플 수
- learning rate : 배치, 에폭에서 모델의 매개변수를 조절하는 비율
    - 값이 작을수록 학습속도가 느리고, 값이 클수록 예측불가능한 동작 발생할 수 있음

In [2]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

## Optimization Loop (최적화 단계)
하나의 epoch는 아래 두 부분으로 구성
- train loop : 학습용 데이터셋을 반복(iterate), 최적 매개변수로 수렴
- validation/ test loop : 모델 성능이 개선되는지 확인을 위해 테스트 데이터셋 반복(iterate)

## loss function (손실함수)
손실함수의 경우 실제값, 예측값 간의 차이를 계산, 이 차이를 최소화 하는것이 중요

In [3]:
# 손실함수 초기화
loss_fn = nn.CrossEntropyLoss()

## Optimizer(최적화)
각 학습단계에서 모델의 오류를 줄이기 위해 매개변수를 조정하는 과정
- 이번 모델에서는 SGD(확률 경사하강법)를 사용
- ADAM, RMSProp와 같은 다양한 Optimizer가 있음

In [4]:
# 학습하려는 모델 매개변수와 학습률 하이퍼파라미터를 등록하여 Optimizer 초기화.
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

학습단계(loop)에서 최적화는 세단계로 이루어짐
- optimizer.zero_grad()를 호출, 모델 매개변수의 변화도를 재설정. 
    - 변화도는 계속 더해지므로 중복계산을 막기 위해 반복할 때마다 0으로 설정.
- loss_backwards()를 호출, 예측손실을 역전파. pytorch는 각 매개변수 손실의 변화도를 저장.
- 변화도 계산 후 optimizer.step()을 호출, 역전파 단계에서 수집된 변화도로 매개변수 조정.

## 전체구현
- train_loop : 최적화 코드를 반복하여 수행
- test_loop : 테스트 데이터로 모델의 성능을 측정

In [5]:
def train_loop(dataloader, model, loss_fn, optimizer):
    # 주어진 dataloader, 모델, 손실함수, optimizer를 사용하여 학습루프 실행
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader): # dataloader에서 미니배치를 가져옴
        
        pred = model(X) # 입력데이터X에 대한 예측
        loss = loss_fn(pred, y) # 예측값과, 실제값(y)간의 손실 계산

        # 역전파
        optimizer.zero_grad() # 이전에 계산된 기울기 초기화
        loss.backward() # 손실에 대한 역전파 수행, 각 파라미터에 대한 기울기 계산
        optimizer.step() # optimizer를 사용, 모델 파라미터 업데이트

        if batch % 100 == 0:
        # 100번째 배치마다 현재 손실, 처리된 데이터 양을 출력
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # 주어진 dataloader, 모델, 손실함수를 사용하여 테스트루프 실행
    size = len(dataloader.dataset) # 전체 데이터셋 크기
    num_batches = len(dataloader) # 데이터로더의 미니배치 개수
    test_loss, correct = 0, 0 # 테스트 손실 및 정확도 초기화

    with torch.no_grad(): # 기울기계산을 비활성화, 테스트 진행
        for X, y in dataloader:
            pred = model(X) # 입력데이터 X에 대한 예측값
            test_loss += loss_fn(pred, y).item() # 테스트 손실을 누적
            # 정확한 예측의 수를 누적
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches # 테스트 손실을 미니배치 수로 나누어 평균 계산
    correct /= size # 정확한 예측 비율을 전체 데이터 크기로 나눔
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

손실함수, optimizer를 초기화 후 train_loop, test_loop에 전달.

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.295934  [   64/60000]
loss: 2.300817  [ 6464/60000]
loss: 2.271741  [12864/60000]
loss: 2.275720  [19264/60000]
loss: 2.251337  [25664/60000]
loss: 2.214800  [32064/60000]
loss: 2.234491  [38464/60000]
loss: 2.198126  [44864/60000]
loss: 2.194066  [51264/60000]
loss: 2.170751  [57664/60000]
Test Error: 
 Accuracy: 44.9%, Avg loss: 2.158278 

Epoch 2
-------------------------------
loss: 2.157417  [   64/60000]
loss: 2.161728  [ 6464/60000]
loss: 2.096771  [12864/60000]
loss: 2.126955  [19264/60000]
loss: 2.066715  [25664/60000]
loss: 1.999630  [32064/60000]
loss: 2.040931  [38464/60000]
loss: 1.961186  [44864/60000]
loss: 1.967473  [51264/60000]
loss: 1.909741  [57664/60000]
Test Error: 
 Accuracy: 59.0%, Avg loss: 1.894164 

Epoch 3
-------------------------------
loss: 1.912306  [   64/60000]
loss: 1.891624  [ 6464/60000]
loss: 1.772518  [12864/60000]
loss: 1.829874  [19264/60000]
loss: 1.708841  [25664/60000]
loss: 1.652295  [32064/600